In [3]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import pyfolio as pf
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import baostock as bs
import pandas as pd
import datetime
from IPython.core.display import HTML
from backtrader_plotting import Bokeh

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
# Import the backtrader platform
import backtrader as bt


def print_top_conv(stock_id, topN=10):
    file_name = './data/'+stock_id+'_day_k_data.csv'
    try:
        df = pd.read_csv(file_name,
                                    header=0,
                                    parse_dates=True,
                                    index_col=0)
        

        # 计算5日内平均增长率
        df['5_day_mean_growth'] = df['close'].pct_change().shift(1).rolling(window=5).mean()

        # 计算5日内最大增长率
        df['5_day_max_growth'] = df['close'].pct_change().shift(1).rolling(window=5).max()

        # 计算5日内最小增长率
        df['5_day_min_growth'] = df['close'].pct_change().shift(1).rolling(window=5).min()

        # 计算5日内价格方差
        df['5_day_price_variance'] = df['close'].shift(1).rolling(window=5).var()
        # 计算5日内价格增长率方差
        df['5_day_price_variance'] = df['close'].pct_change().shift(1).rolling(window=5).var()
        # 计算5日内成交价的4次方导数
        df['5-Day Price 4th Derivative'] = df['close'].shift(1).diff(periods=4)

        # 计算5日内价格与成交量的乘积
        df['5-Day Price x Volume'] = df['close'] * df['volume']

        # 计算3日后价格是否增长超过3%，生成输出信号
        df['Signal'] = (df['close'].shift(-3) / df['close'] > 1.03).astype(int)
        
                # 计算每个部分的长度
        total_length = len(df)
        part_length = total_length // 3  # 三个部分的长度
        df_part1 = df.iloc[:part_length]
        df_part2 = df.iloc[part_length:2*part_length]
        df_part3 = df.iloc[2*part_length:]
        df_partAll = df

        # 计算每个部分的相关性矩阵
        correlation_matrix1 = df_part1.drop(['tradestatus', 'isST'], axis=1).corr(method='kendall')
        correlation_matrix2 = df_part2.drop(['tradestatus', 'isST'], axis=1).corr(method='kendall')
        correlation_matrix3 = df_part3.drop(['tradestatus', 'isST'], axis=1).corr(method='kendall')
        correlation_matrixAll = df_partAll.drop(['tradestatus', 'isST'], axis=1).corr(method='kendall')
        # 找到与'Signal'列相关性系数最高的5项的列名
        top_feature_names1 = correlation_matrix1['Signal'].sort_values(ascending=False).head(topN)

        # 打印相关性系数最高的5项的列名
        print(f'股票ID:{stock_id}')
        #for feature, correlation in top_feature_names1.items():
            #print(f'特征1: {feature}, 相关性系数: {correlation}')
        top_feature_names2 = correlation_matrix2['Signal'].sort_values(ascending=False).head(topN)

        # 打印相关性系数最高的5项的列名
        #for feature, correlation in top_feature_names2.items():
            #print(f'特征2: {feature}, 相关性系数2: {correlation}')
        top_feature_names3 = correlation_matrix3['Signal'].sort_values(ascending=False).head(topN)

        # 打印相关性系数最高的5项的列名
        #for feature, correlation in top_feature_names3.items():
            #print(f'特征3: {feature}, 相关性系数3: {correlation}')

        selected_features  = correlation_matrixAll['Signal'].sort_values(ascending=False).head(5).index
        X = df[selected_features]
        X['Signal'] = df['Signal']
        X.dropna(inplace=True)
        y = X['Signal']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        models = {
            'Linear Regression': LinearRegression(),
            'Decision Tree Regressor': DecisionTreeRegressor(),
            'Random Forest Regressor': RandomForestRegressor(),
            'Support Vector Regressor': SVR(),
            'K-Nearest Neighbors Regressor': KNeighborsRegressor()
        }

        # 训练模型并评估性能
        for model_name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            print(f'{model_name} - 均方误差 (MSE): {mse}')
    except FileNotFoundError:
        # 处理文件不存在的异常
        print("文件不存在，无法读取数据")
print_top_conv('sz.000333')
print_top_conv('sh.600887')
print_top_conv('sz.000651')
print_top_conv('sz.002594')

股票ID:sz.000333
Linear Regression - 均方误差 (MSE): 5.029284344843997e-28
Decision Tree Regressor - 均方误差 (MSE): 0.0
Random Forest Regressor - 均方误差 (MSE): 0.0
Support Vector Regressor - 均方误差 (MSE): 0.1775236344622928


d:\program files (x86)\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\program files (x86)\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


K-Nearest Neighbors Regressor - 均方误差 (MSE): 0.19000000000000003
股票ID:sh.600887
Linear Regression - 均方误差 (MSE): 8.036058524766043e-29
Decision Tree Regressor - 均方误差 (MSE): 0.0
Random Forest Regressor - 均方误差 (MSE): 0.0
Support Vector Regressor - 均方误差 (MSE): 0.1618620183371709
K-Nearest Neighbors Regressor - 均方误差 (MSE): 0.19037288135593222


d:\program files (x86)\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\program files (x86)\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


股票ID:sz.000651
Linear Regression - 均方误差 (MSE): 9.673231280910311e-29
Decision Tree Regressor - 均方误差 (MSE): 0.0
Random Forest Regressor - 均方误差 (MSE): 0.0
Support Vector Regressor - 均方误差 (MSE): 0.18329237847809887


d:\program files (x86)\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\program files (x86)\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


K-Nearest Neighbors Regressor - 均方误差 (MSE): 0.1933783783783784
股票ID:sz.002594
Linear Regression - 均方误差 (MSE): 1.655216839757269e-28
Decision Tree Regressor - 均方误差 (MSE): 0.0
Random Forest Regressor - 均方误差 (MSE): 0.0


d:\program files (x86)\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\program files (x86)\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Support Vector Regressor - 均方误差 (MSE): 0.22079007173435772
K-Nearest Neighbors Regressor - 均方误差 (MSE): 0.22891891891891894


In [4]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
plt.figure(figsize=(30, 30),dpi=250)
sns.heatmap(correlation_matrix, square=True, linewidths=.5, annot=True)
plt.savefig('./Image.png')

NameError: name 'correlation_matrix' is not defined

<Figure size 7500x7500 with 0 Axes>

In [40]:
# 找到与'Signal'列相关性系数最高的5项的列名
top_feature_names = correlation_matrix['Signal'].sort_values(ascending=False).head(10).index.tolist()

# 打印相关性系数最高的5项的列名
print(top_feature_names)

['Signal', 'volume', 'turn', 'amount', '5-Day Price x Volume', '5_day_price_variance', 'high', 'open', 'close', 'preclose']


In [7]:
df2 = pd.read_csv('./data/sz.002373_day_k_data.csv',
                            header=0,
                            parse_dates=True,
                            index_col=0)

In [12]:
df2['open'].pct_change()

date
2017-01-03         NaN
2017-01-04    0.002896
2017-01-05    0.017329
2017-01-06    0.007807
2017-01-09    0.003521
                ...   
2023-02-07   -0.013876
2023-02-08    0.011257
2023-02-09   -0.025974
2023-02-10    0.023810
2023-02-13   -0.011163
Name: open, Length: 1484, dtype: float64

In [13]:
df2['open'].pct_change().shift(1)

date
2017-01-03         NaN
2017-01-04         NaN
2017-01-05    0.002896
2017-01-06    0.017329
2017-01-09    0.007807
                ...   
2023-02-07    0.021739
2023-02-08   -0.013876
2023-02-09    0.011257
2023-02-10   -0.025974
2023-02-13    0.023810
Name: open, Length: 1484, dtype: float64